In [ ]:
# %% [code]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os

from src.data_processing import load_config, load_data, preprocess_data, sample_data

# Configuration visuelle
sns.set(style="whitegrid", context="talk")
%matplotlib inline

# Charger la configuration
config = load_config("config/config.yaml")
print("Configuration chargée avec succès.")
